# Kolmogorov Complexity to Compress DNA Sequences Using Python
Long form blogpost can be found over here: [yacinemahdid.com](yacinemahdid.com)

The data used can be found over here: [Kaggle/Humans.txt file](https://www.kaggle.com/nageshsingh/dna-sequence-dataset/version/1?select=human.txt)

In [7]:
import pandas as pd
HUMAN_DNA_DATASET = "C:/Users/Yacin/Desktop/human.txt"
df = pd.read_csv(HUMAN_DNA_DATASET, sep = "\t", names = ['sequence','class'], skiprows=1)

# No idea what the class variable is though as it wasn't mentioned in the dataset on Kaggle.
df

,sequence,class
0,ATGCCCCAACTAAATACTACCGTATGGCCCACCATAATTACCCCCA...,4
1,ATGAACGAAAATCTGTTCGCTTCATTCATTGCCCCCACAATCCTAG...,4
2,ATGTGTGGCATTTGGGCGCTGTTTGGCAGTGATGATTGCCTTTCTG...,3
3,ATGTGTGGCATTTGGGCGCTGTTTGGCAGTGATGATTGCCTTTCTG...,3
4,ATGCAACAGCATTTTGAATTTGAATACCAGACCAAAGTGGATGGTG...,3
...,...,...
4375,ATGGAAGATTTGGAGGAAACATTATTTGAAGAATTTGAAAACTATT...,0
4376,ATGCAGTCCTTTCGGGAGCAAAGCAGTTACCACGGAAACCAGCAAA...,6
4377,ATGCAGTCCTTTCGGGAGCAAAGCAGTTACCACGGAAACCAGCAAA...,6
4378,ATGGGGCACCTGGTTTGCTGTCTGTGTGGCAAGTGGGCCAGTTACC...,6
